In [8]:
%%writefile ../README.md
# Fetch GPX-Tracks from EuroVelo-Website

We obtain the data from:

```text
https://en.eurovelo.com/#
```

On the website the there is a `GET` method hidden, i.e. `https://en.eurovelo.com/route/get-gpx/<id>`. Unfortunately, so far I couldn't find a list with the IDs. Hence, in the following we took the brute force approach and tried IDs from 0 to N.


Regarding the GPX data wrangling, we took inspiration from:

* https://thatmaceguy.github.io/python/gps-data-analysis-intro/
* https://towardsdatascience.com/data-science-for-cycling-how-to-read-gpx-strava-routes-with-python-e45714d5da23
* https://towardsdatascience.com/visualizing-geospatial-data-in-python-e070374fe621
* https://towardsdatascience.com/simple-gps-data-visualization-using-python-and-open-street-maps-50f992e9b676
* https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972

(TODO) For the interaction with Garmin Connect:

* https://github.com/matin/garth
* https://github.com/cyberjunky/python-garminconnect
* https://github.com/cpfair/tapiriik/blob/ee0153818981863e19750d59727d77363e5a37fe/tapiriik/services/GarminConnect/garminconnect.py#L548
* https://medium.com/@rama_m/how-to-download-garmin-training-data-using-python-api-5e94488a2317
* https://github.com/Pythe1337N/garmin-connect
* https://github.com/abrander/garmin-connect
* https://developer.garmin.com/gc-developer-program/activity-api/

Overwriting ../README.md


In [9]:
# ! pip install -r ../requirements.txt

In [10]:
import httpx
import gpxpy

from pathlib import Path
from tqdm import tqdm
from pprint import pprint
from itertools import compress

from functools import lru_cache
from gpxpy.gpx import GPX

In [11]:
URL = "https://en.eurovelo.com/route/get-gpx"
SAVE_FOLDER = Path("./../routes").expanduser().resolve()
MAX_RANGE_IDS = 1_000
DEV = True

In [12]:
@lru_cache
def parse_gpx_data(xml: str) -> GPX:
    return gpxpy.parse(xml)

In [5]:
unsuccessful_requests = []
for i in tqdm(range(MAX_RANGE_IDS)):
    if i > 3 and DEV:
        break
    _url = "/".join((URL, f"{i:03d}"))
    _request = httpx.get(_url)
    try:
        _request.raise_for_status()
    except Exception as ex:
        unsuccessful_requests.append({"url": _url, "stage": "request", "error": ex})
        continue
    _content = _request.content.decode()
    try:
        gpx_data = parse_gpx_data(_content)
    except Exception as ex:
        unsuccessful_requests.append({"url": _url, "stage": "gpx parsing", "error": ex})
        continue
    (SAVE_FOLDER / f"file_{i:03d}.gpx").write_text(_content)
len(unsuccessful_requests)

  0%|          | 4/1000 [00:15<1:05:55,  3.97s/it]


2

In [13]:
# list all the routes which go through Berlin
_CITIES = ("Berlin", "Dresden")
for _file in SAVE_FOLDER.iterdir():
    if _file.is_dir():
        continue
    gpx_data = parse_gpx_data(_file.read_text())
    _track_name_containing_city = {
        _city: [_city.lower() in _track.name.lower() for _track in gpx_data.tracks]
        for _city in _CITIES
    }
    if all(
        any(_track_name_contains_city)
        for _track_name_contains_city in _track_name_containing_city.values()
    ):
        print(
            f"The following file contains tracks which is going through {_CITIES}: {_file}."
        )
        for _city, _track_name_filter in _track_name_containing_city.items():
            print(f"The tracks withing the route which mention {_city} are:")
            pprint(
                [
                    (_track.name, _track.description)
                    for _track in compress(gpx_data.tracks, _track_name_filter)
                ]
            )
        print("=" * 77)

The following file contains tracks which is going through ('Berlin', 'Dresden'): /home/christoph/MachineLearningDeepLearning/euro_velo_routes/routes/file_146.gpx.
The tracks withing the route which mention Berlin are:
[('059: Hennigsdorf – Berlin (DEVELOPED)', 'DEVELOPED'),
 ('060: Berlin – Potsdam (DEVELOPED)', 'DEVELOPED')]
The tracks withing the route which mention Dresden are:
[('065: Riesa – Dresden (DEVELOPED)', 'DEVELOPED'),
 ('066: Dresden – Decin (DEVELOPED)', 'DEVELOPED')]
The following file contains tracks which is going through ('Berlin', 'Dresden'): /home/christoph/MachineLearningDeepLearning/euro_velo_routes/routes/file_030.gpx.
The tracks withing the route which mention Berlin are:
[('059: Hennigsdorf – Berlin (DEVELOPED)', 'DEVELOPED'),
 ('060: Berlin – Potsdam (DEVELOPED)', 'DEVELOPED')]
The tracks withing the route which mention Dresden are:
[('065: Riesa – Dresden (DEVELOPED)', 'DEVELOPED'),
 ('066: Dresden – Decin (DEVELOPED)', 'DEVELOPED')]


In [14]:
# Prepare GPX-tracks for imports to Garmin
_prefix = "EuroVelo7"
_filename = "file_030.gpx"  # "file_146.gpx"

gpx_data = parse_gpx_data((_file_of_origin := SAVE_FOLDER / _filename).read_text())
_folder_for_tracks = (
    _file_of_origin.parent
    / f"tracks_from_{_file_of_origin.stem}_prepared_for_imports_to_garmin_connect"
)
if not _folder_for_tracks.exists():
    _folder_for_tracks.mkdir()
_cloned_gpx_data = gpx_data.clone()
for i, _track in tqdm(enumerate(gpx_data.tracks, start=1)):
    _cloned_gpx_data.tracks.clear()
    _cloned_gpx_data.tracks.append(_track.clone())
    _cloned_gpx_data.tracks[-1].name = " | ".join((_prefix, _track.name))
    (_folder_for_tracks / f"track_{i:03d}.gpx").write_text(_cloned_gpx_data.to_xml())

136it [00:03, 34.53it/s]
